In [11]:
import numpy as np
import Code
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import random
import plotly.graph_objects as go

## <span style="color:orange">  **Cadre général de la descente de gradient stochastique pour obtenir une estimation du maximum de vraisemblance**  </span>

On se donne un échantillon d'observations $\boldsymbol{x} \in \mathbb{R}^{20} = [x^{(1)}, ..., x^{(20)}]^T$ tiré selon une loi paramétrée par $\theta$, qui est donc le paramètre du modèle probabiliste que nous souhaitons estimer. La log-vraisemblance des données observées $\ell (\theta) = \log p_{\theta}(\boldsymbol{x}) = \sum_{i=1}^{20} \log p_{\theta}(x^{(i)})$ est maximisée pour obtenir une estimation de maximum de vraisemblance (EMV) du paramètre $\theta$. 

En utilisant la **descente de gradient stochastique (SGD)**, l'algorithme pour estimer les paramètres peut être formulé comme suit :


1. **Initialisation** : On choisit un $\theta^{(0)}$ De manière aléatoire ou déterministe. On peut par exemple choisir une telle valeur à partir d'une première approximation.

2. **Itération** : À chaque itération $t$, les paramètres $\theta$ sont mis à jour en utilisant un échantillon aléatoire de données $x^{(i)}$ (donc pour $i \in \{1, ..., 20\}$) tiré de l'ensemble de données et on a alors : 
\begin{equation}
\theta^{(t+1)} = \theta^{(t)} - \eta \nabla_{\theta} \log p_{\theta^{(t)}}(x^{(i)}),
\end{equation}

    où $\eta$ est le **taux d'apprentissage** (également appelé pas d'apprentissage) et $\nabla_{\theta} \log p_{\theta^{(t)}}(\boldsymbol{x}^{(i)})$ est le gradient du logarithme de la vraisemblance conditionnelle évalué pour l'échantillon $\boldsymbol{x}^{(i)}$ à l'itération $t$. 

3. **Convergence** : On répète $(1)$ jusqu'à ce qu'un critère d'arrêt prédéfini soit satisfait. Par exemple, on peut fixer pour un $\epsilon$ donné le critère d'arrêt suivant :  
$$ 
\| \theta^{(t+1)} - \theta^{(t)} \| < \epsilon
$$



### <span style="color:violet">  1.1. **Calcul explicite du gradient de la log-vraisemblance**  </span>

Dans l'article, pour effectuer l'expérience empirique, on nous donne la valeur explicite de la densité avec laquelle on tire les observations $\boldsymbol{x}$. On nous donne : 

$$
p_{\theta}(\boldsymbol{x}) = \mathcal{N}(\boldsymbol{x}|\theta \mathbf{1}_{20}, 2\mathbf{I}_{20}) = \frac{1}{(4\pi)^{10}} \exp \left\{ -\frac{1}{4} (\boldsymbol{x} - \theta \mathbf{1}_{20})^T  (\boldsymbol{x} - \theta \mathbf{1}_{20}) \right\}
$$

Ainsi, on a : 

$$
\log p_{\theta}(\boldsymbol{x}) = \sum_{i=1}^{20}  \log p_{\theta}(x^{(i)}) = \sum_{i=1}^{20} \log \left( \frac{1}{2\sqrt{\pi}} \exp\left(-\frac{(x - \theta)^2}{4}\right) \right)
$$

Une fois cette expression explicite de la log-vraisemblance obtenue, on peut calculer explicitement le gradient de celle-ci **par rapport au paramètre  $\theta$**. On a : 

$$
\nabla_{\theta} \log p_{\theta} (\boldsymbol{x}) = \sum_{i=1}^{20}  \log \nabla_{\theta} p_{\theta}(x^{(i)}) = \frac{1}{2} \sum_{i=1}^{20} (x^{(i)} - \theta)
$$

Si bien que l'on retrouve la version du gradient **en un point** de l'échantillon $\boldsymbol{x}$ que l'on utilise dans l'étape itérative : 

$$
\nabla_{\theta} \log p_{\theta}(x^{(i)}) = \frac{1}{2} (x^{(i)} - \theta)
$$

A MODIFIER + ERREUR D

### <span style="color:violet">  1.2. **Tirage du paramètre génératif $\theta$**  </span>

Avant toute chose, on tire une valeur du paramètre $\theta$ que l'on considère comme étant la quantité que l'on souhaite approximer dans la suite de cette partie. On sait d'après le cadre de l'application numérique de l'article que $\theta \sim \mathcal{N}(1, 0)$. 

In [5]:
#theta_true = np.random.multivariate_normal(np.zeros(20), np.identity(20))
theta_true = np.random.normal(0, 1)
print(f"La valeur de theta à estimer est {theta_true}")

La valeur de theta à estimer est 0.022006044445536263


### <span style="color:violet">  1.3. **Tirage de l'échantillon**  </span>

In [6]:
def p_theta_x(theta_val):
    
    x = []

    for _ in range(20):
        x.append(np.random.normal(loc=theta_val, scale=np.sqrt(2)))

    return x

true_x = p_theta_x(theta_true)

print(f"L'échantillon d'observation x de taille 20 sur lequel on travaillera tiré est \n \n {true_x}")

L'échantillon d'observation x de taille 20 sur lequel on travaillera tiré est 
 
 [1.0998730040343296, 0.27054871761271515, 1.1766033936922793, 2.03807408512119, 0.2720561737234926, -1.646616398825939, -1.0797569607244144, -0.756181732251176, -0.7921305436187636, -1.4223885417711613, -2.1801152106167634, 2.043080982354381, -0.9158115622340559, -1.3438090997542205, 2.3043739214296153, -0.39419692309662957, -1.3829457637212934, -0.6836905305034041, -1.59459302552407, -0.42190247080279475]


### <span style="color:violet">  1.4. **Adaptation de l'itération avec des estimation du gradient**  </span>

 l'idée est désormais d'adapter l'étape itérative décrite précédemment. Dans le cas où celle-ci est mise à jour à partir d'une estimation du gradient. On notera $\hat{\nabla}^{\text{SUMO}}_{\theta}, \hat{\nabla}^{\text{ML-SS}}_{\theta}, \hat{\nabla}^{\text{ML-RR}}_{\theta}$ et $\hat{\nabla}^{\text{IWAE}}_{\theta}$ l'estimateur du gradient selon les quatre méthodes sur lesquels repose nous travail (SUMO, ML_SS, ML_RR et IWAE).

**Itération** : Par exemple, en utilisant l'estimateur du gradient avec la méthode SUMO, on a l'itération $t$ du paramètres $\theta$ donnée, en utilisant un échantillon aléatoire de données $x^{(i)}$ (donc pour $i \in \{1, ..., 20\}$) tiré de l'ensemble de données : 
\begin{equation}
\theta^{(t+1)} = \theta^{(t)} - \eta \hat{\nabla}^{\text{SUMO}}_{\theta} (x^{(i)}),
\end{equation} 

A MODIFIER (?)

### <span style="color:violet">  1.4. **Étape itérative avec des estimation du gradient**  </span>

In [7]:
def SGD_iteration(theta_t, true_x, eta, r, k_IWAE, n_simulations, method):

    ## on tire un indice entre 0 et 19 (inclus)
    random_index = random.randint(0, 19)

    ## On crée un vecteur de taille 20 dont tous les éléments sont nuls, sauf celui donnée par l'index random_index
    modified_x = np.zeros(20)
    modified_x[random_index] = true_x[random_index]

    ## on recalcule noised_A et noised_b SELON LE PARAMÈTRE PRÉCÉDENT theta_t
    A, b = np.eye(20), (theta_t/2)*np.ones(20)
    noised_A, noised_b = Code.noised_params(A, b)

    if method == 'IWAE':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_IWAE(modified_x, noised_A, noised_b, theta_t, k_IWAE, n_simulations)

    elif method == 'SUMO':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_SUMO(r, modified_x, noised_A, noised_b, theta_t, n_simulations)

    elif method == 'ML_SS':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_IWAE(r, modified_x, noised_A, noised_b, theta_t, n_simulations)

    elif method == 'ML_RR':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_IWAE(r, modified_x, noised_A, noised_b, theta_t, n_simulations)

    theta_next =  theta_t - eta * grad_theta_t

    return theta_next

In [8]:
def SGD_iteration_tom(true_x, sum_log, position, theta_t, eta, r, k_IWAE, n_simulations, method):

    #modified_x = true_x[position]

    ## on recalcule noised_A et noised_b SELON LE PARAMÈTRE PRÉCÉDENT theta_t
    #A, b = np.eye(20), (theta_t/2)*np.ones(20)
    #noised_A, noised_b = Code.noised_params(A, b)
    
    A, b = np.eye(2), (theta_t/2)*np.ones(2)
    noised_A, noised_b = Code.noised_params(A, b, dim=2)

    if method == 'IWAE':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_IWAE(true_x, noised_A, noised_b, float(theta_t), k_IWAE, n_simulations, dim = 2, one_shot=True)

    elif method == 'SUMO':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_SUMO(r, true_x, noised_A, noised_b, float(theta_t), n_simulations, dim = 2, one_shot=True)

    elif method == 'ML_SS':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_ML_SS(r, true_x, noised_A, noised_b, float(theta_t), n_simulations, dim = 2, one_shot=True)

    elif method == 'ML_RR':
        ## on calcule le gradient selon l'estimateur donné dans method
        grad_theta_t = Code.grad_ML_RR(r, true_x, noised_A, noised_b, float(theta_t), n_simulations, dim = 2, one_shot=True)

     ## On prend le ième terme de notre somme et cela sera celui-ci que l'on dérive à cette itération
    sum_log[position] = grad_theta_t

    theta_next =  theta_t + eta * sum(sum_log)/len(sum_log) ## ON est sur du gradient ascent

    return theta_next, sum_log

Une fois le code pour les itération fini, on s'attaque à la convergence de l'algorithme de descendre de gradient stochastique rien

In [9]:
def SGD_convergence(theta_0, true_x, eps, eta, r, k_IWAE, n_simulations, method):
    
    ## On initialise la somme des log pour chaque i
    sum_log = []

    if method == 'IWAE':
        
        for i in range(10) : 
            sum_log.append(Code.grad_IWAE([true_x[2*i], true_x[2*i+1]], np.eye(2), (theta_0/2)*np.ones(2), theta_0, k_IWAE, n_simulations, dim = 2, one_shot=True, range=1))

    elif method == 'SUMO':

        for i in range(10) : 
            sum_log.append(Code.grad_SUMO(r, [true_x[2*i], true_x[2*i+1]], np.eye(2), (theta_0/2)*np.ones(2), theta_0, n_simulations, dim = 2, one_shot=True, range=1))

    elif method == 'ML_SS':

        for i in range(10) : 
            sum_log.append(Code.grad_ML_SS(r, [true_x[2*i], true_x[2*i+1]], np.eye(2), (theta_0/2)*np.ones(2), theta_0, n_simulations, dim = 2, one_shot=True, range=1))
    
    elif method == 'ML_RR':

        for i in range(10):
            sum_log.append(Code.grad_ML_RR(r, [true_x[2*i], true_x[2*i+1]], np.eye(2), (theta_0/2)*np.ones(2), theta_0, n_simulations, dim = 2, one_shot=True, range=1))

    #print(f" voici la sommelog {sum_log}")

    theta_t = [0, theta_0]
    #theta_t.append(SGD_iteration_tom(true_x[], sum_log, 0, theta_0, eta, r, k_IWAE, n_simulations, method)[0])
    t = 1
    #while abs(theta_t[t] - theta_t[t-1]) > eps:
    while abs(theta_t[t] - theta_true) > eps:
        
        ## peut-être que l'on peut inverser les deux boucles cela ferait plus de sens mais c'est pour voir si ça s'arrête

        for i in range(10):
            
            theta_next, sum_log = SGD_iteration_tom([true_x[2*i], true_x[2*i+1]], sum_log, i, theta_t[t], eta, r, k_IWAE, n_simulations, method)
            
            theta_t.append(theta_next)
            #print(theta_next)
            if abs(theta_next - theta_true) < eps:
                return theta_t
            else : 
                t += 1

    return theta_t

In [10]:
theta_0 = -3
#eta = 0.0001
#eps = 0.001
eta = 0.1
eps = 0.01
r = 0.6
k_IWAE = 5
n_simulations = 2

methodes = ['IWAE', 'SUMO', 'ML_SS', 'ML_RR']
SGD_conv = [[], [], [], []]

for method in methodes : 

    SGD_conv[methodes.index(method)] = SGD_convergence(theta_0, true_x, eps, eta, r, k_IWAE, n_simulations, method)

 voici la sommelog [0.15013421636307722, -0.42969640348492, 0.4516090045139629, 0.39903446107084317, 0.6489191977303634, 0.5707332024708398, -0.21996134932830325, 0.9370677010310029, 0.3873169514712629, -0.3660491579746141]
 voici la sommelog [-0.03716251136460169, -1.0570790743870209, -0.24772179280215623, -0.6907493901430928, -0.06872029970918403, -3.9626374362330625, 0.3552767783581965, -0.31528371451260107, -0.13381097320254165, 1.0814775048914953]
 voici la sommelog [0.17502699743986838, -0.16395885345862382, -0.0006510603520988312, -0.167553872372721, 0.04618348800027228, -1.4685044548635071, -0.21276143513169155, 0.9074203380776051, 0.19951340840304965, 0.2930538534181797]
 voici la sommelog [0.04538326032071005, 1.0682216319266304, -0.6274689933747659, 0.1613079906037862, 0.3040235241832794, 2.029619384866787, -0.10425323726492808, 1.1512689155169262, -0.07103322302366788, -0.2659391956364887]


In [21]:
def plot_convergence(convergence_values, theta_true):
    """
    Trace la convergence des points vers la valeur du vrai paramètre.

    Args:
    - convergence_values (list or numpy array): Liste des valeurs de convergence de l'algorithme.
    - theta_true (float): Valeur vraie du paramètre.

    """
    num_iterations = max(len(convergence_values[i]) for i in range(4))  # Nombre d'itérations
    iterations = np.arange(1, num_iterations + 1)  # Numéro des itérations

    # Création de la figure
    fig = go.Figure()

    for i, conv_values in enumerate(convergence_values):
        fig.add_trace(go.Scatter(x=iterations, y=conv_values, mode='markers', name=methodes[i], marker=dict(color=['purple', 'orange', 'green', 'yellow'][i], symbol='x')))
    
    fig.add_shape(type='line', x0 = iterations, x1 = iterations, y0=theta_true, y1 = theta_true, line=dict(color='black', dash='dash'))

    
    # Mise en forme de la figure
    fig.update_layout(
        xaxis=dict(title='Valeur de convergence'),
        yaxis=dict(title='Itérations'),
        title=f'Comparaison de la convergence de la SGD avec les différents estimateurs du gradient',
        legend=dict(x=0, y=1, traceorder='normal', font=dict(size=12)),
        showlegend=True
    )

    # Affichage de la figure
    fig.show()


In [22]:
plot_convergence(SGD_conv, theta_true)

La descente de gradient stochastique (SGD) est un algorithme d'optimisation largement utilisé pour estimer les paramètres d'un modèle probabiliste par maximum de vraisemblance. L'algorithme fonctionne de la manière suivante :

1. Choisir un point initial $ \theta_0 $ dans l'espace des paramètres.
2. Itération jusqu'à convergence :
2.1. Échantillonnage stochastique : Sélectionner aléatoirement un échantillon $(x_i, y_i)$ parmi les données d'entraînement.
2.2. Calcul du gradient : Calculer le gradient de la fonction de perte par rapport aux paramètres du modèle en utilisant l'échantillon sélectionné : $ \nabla_{\theta} \mathcal{L}(\theta; x_i, y_i) $.
2.3. Mise à jour des paramètres : Mettre à jour les paramètres du modèle en utilisant le gradient calculé et un taux d'apprentissage $ \eta $ : $ \theta_{t+1} = \theta_t - \eta \nabla_{\theta} \mathcal{L}(\theta_t; x_i, y_i) $.


\textbf{Remarques :}
\begin{itemize}
    \item Le terme "stochastique" vient du fait que le gradient est calculé sur un seul échantillon à la fois, ce qui rend l'algorithme adapté aux grands ensembles de données.
    \item Le choix du taux d'apprentissage \( \eta \) est crucial ; un taux d'apprentissage trop grand peut entraîner des oscillations et une convergence instable, tandis qu'un taux d'apprentissage trop petit peut ralentir la convergence.
    \item La convergence de l'algorithme dépend de plusieurs facteurs, notamment du choix du taux d'apprentissage, de la fonction de perte utilisée et de la nature de l'ensemble de données.
\end{itemize}

